In [1]:

import sys
import os

import ccxt
import backtesting as bt
from backtesting import Backtest, Strategy
import talib
import pandas_ta as ta

import pandas as pd
import numpy as np

In [2]:
def get_data(symbol, timeframe, limit):
    exchange = ccxt.kraken()
    exchange.load_markets()

    data = exchange.fetch_ohlcv('BTC/USDT', timeframe=timeframe, limit=limit)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']).rename(columns={
        'timestamp': 'timestamp',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

In [3]:
data = get_data('BTC/USDT', '1h', 1000)

In [4]:
data

,Open,High,Low,Close,Volume
timestamp,,,,,
2024-04-20 03:00:00,63845.1,64009.1,63806.5,63967.9,15.782354
2024-04-20 04:00:00,63959.7,64217.0,63862.1,64142.0,0.493089
2024-04-20 05:00:00,64126.3,64217.0,63984.9,64151.8,3.306415
2024-04-20 06:00:00,64145.9,64211.7,63979.5,64056.2,2.593212
2024-04-20 07:00:00,64079.9,64158.7,63987.2,63987.3,1.146660
...,...,...,...,...,...
2024-05-19 22:00:00,66280.1,66383.5,66268.1,66347.6,0.705564
2024-05-19 23:00:00,66347.6,66347.6,66266.0,66290.2,3.082073
2024-05-20 00:00:00,66290.1,66436.0,66171.0,66435.9,2.881712


# EMA strategy

In [11]:
class EMA_strategy(Strategy):
    ema_a = 3

    def init(self):
        """ define the indicators you need """
        
        self.line_a = self.I(lambda df, length: df.ta.ema(close = df['Close'], length=length), self.data.df, self.ema_a, name='line_a')
        # self.myother_indicator = self.I(....)

    def next(self):
        """ decide when to buy or sell """
        
        
        if self._broker.orders:
            return

        """
        buy: when the lastest close price is above the EMA
        
        sell: when the lastest close price is below the EMA
        """
        

        if self.data.Close[-1] > self.line_a[-1]:
            self.buy()
            
        elif self.data.Close[-1] < self.line_a[-1]:
            self.sell()
            
            

In [12]:
bt = Backtest(
    data, 
    EMA_strategy,
    commission=0.00075, 
    cash=100000, 
    )

strat = bt.run() 
bt.plot()

Row(id='3183', ...)

In [7]:
pd.DataFrame(strat)

,0
Start,2024-04-20 03:00:00
End,2024-05-20 02:00:00
Duration,29 days 23:00:00
Exposure Time [%],91.527778
Equity Final [$],97898.514
Equity Peak [$],105240.514425
Return [%],-2.101486
Buy & Hold Return [%],4.14708
Return (Ann.) [%],-22.125367
Volatility (Ann.) [%],21.942402


## Optimization

In [8]:
import multiprocessing as mp
mp.set_start_method('fork') 

In [8]:
bt = Backtest(
    data, 
    EMA_strategy,
    commission=0.00075, 
    cash=100000, 
    )

stats = bt.optimize(
                    ema_a=range(5, 70, 1),
                    ema_b=range(5, 70, 1),
                    constraint=lambda param: param.ema_a < param.ema_b,
                    maximize='Sharpe Ratio',
                    max_tries=500,
                    random_state=1) 

pd.DataFrame(stats)
stats._strategy

/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 501 configurations.
  output = _optimize_grid()
/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/11 [00:00<?, ?it/s]

<Strategy EMA_strategy(ema_a=16,ema_b=21)>

In [9]:
bt = Backtest(
    data, 
    EMA_strategy,
    commission=0.00075, 
    cash=100000, 
    )

strat = bt.run(ema_a=16, ema_b=21)
bt.plot()

Row(id='2784', ...)

In [10]:
strat

Start                     2024-04-20 03:00:00
End                       2024-05-20 02:00:00
Duration                     29 days 23:00:00
Exposure Time [%]                   96.944444
Equity Final [$]                  107394.1213
Equity Peak [$]                  108256.73945
Return [%]                           7.394121
Buy & Hold Return [%]                 4.14708
Return (Ann.) [%]                  131.617041
Volatility (Ann.) [%]               56.484077
Sharpe Ratio                         2.330162
Sortino Ratio                       11.048115
Calmar Ratio                        27.012888
Max. Drawdown [%]                    -4.87238
Avg. Drawdown [%]                   -1.078603
Max. Drawdown Duration       13 days 18:00:00
Avg. Drawdown Duration        1 days 15:00:00
# Trades                                   18
Win Rate [%]                        44.444444
Best Trade [%]                       7.233068
Worst Trade [%]                     -3.470266
Avg. Trade [%]                    

In [11]:
strat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,22,107,64814.374350,66031.9,1217.525650,0.018785,2024-04-21 01:00:00,2024-04-24 14:00:00,3 days 13:00:00
1,-1,107,191,65982.376075,63599.0,2383.376075,0.036121,2024-04-24 14:00:00,2024-04-28 02:00:00,3 days 12:00:00
2,1,191,213,63646.699250,63106.3,-540.399250,-0.008491,2024-04-28 02:00:00,2024-04-29 00:00:00,0 days 22:00:00
3,-1,213,236,63058.970275,63872.4,-813.429725,-0.012900,2024-04-29 00:00:00,2024-04-29 23:00:00,0 days 23:00:00
4,1,236,247,63920.304300,61702.1,-2218.204300,-0.034703,2024-04-29 23:00:00,2024-04-30 10:00:00,0 days 11:00:00
5,-1,247,301,61655.823425,59269.9,2385.923425,0.038697,2024-04-30 10:00:00,2024-05-02 16:00:00,2 days 06:00:00
6,1,301,397,59314.352425,63604.6,4290.247575,0.072331,2024-05-02 16:00:00,2024-05-06 16:00:00,4 days 00:00:00
7,-1,397,415,63556.896550,64127.2,-570.303450,-0.008973,2024-05-06 16:00:00,2024-05-07 10:00:00,0 days 18:00:00
8,1,415,419,64175.295400,63197.2,-978.095400,-0.015241,2024-05-07 10:00:00,2024-05-07 14:00:00,0 days 04:00:00
9,-1,419,421,63149.802100,63757.3,-607.497900,-0.009620,2024-05-07 14:00:00,2024-05-07 16:00:00,0 days 02:00:00
